In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 코랩에 구글드라이브 연동해서 안에 파일들을 사용할 수 있게끔 해준다

Mounted at /content/drive


In [ ]:
import os

os.chdir('drive/MyDrive/Multimodal_Fusion_Ratings')
print(os.listdir())

['Multi-Modal_Deep_Learning_for_Credit_Rating_Prediction_Using_Text_and_Numerical_Data_Streams.pdf', 'MultimodalFusionRatings', 'Group4.ipynb']


In [ ]:
# 수집한 데이터 load

import pandas as pd
data_main=pd.read_excel("MultimodalFusionRatings/Data/Artificial_Data.xlsx")

In [ ]:
data_main['Rating']

0       A+
1       BB
2       BB
3      BBB
4       AA
      ... 
95    BBB+
96      AA
97      BB
98     AA+
99     BBB
Name: Rating, Length: 100, dtype: object

In [ ]:
# 구글 코랩에서 진행 예정이라 따로 memory growth를 직접 설정할 필요는 없음.
# 진행하면서 memory allocation 문제가 생기면 그때 한번 확인하고 돌려보기

# import tensorflow as tf
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)
# Set multiple gpus
# strategy = tf.distribute.MirroredStrategy()

In [26]:
#change of the rating classes to new classes
import numpy as np



# TODO: 실제 데이터에서는 이부분에 moody's rating은 다르기 때문에,
# 딕셔너리를 따로 만들어서 일단 하나로 통합을 하고 변환을 해줘야함
# 실제 데이터가 어떤지 받아봐야 확인 가능



lbl_first=data_main["Rating"].values # rating을 수정해주기 전 데이터를 보관 (.values 는 np array로 받게해줌)
lbl_final=lbl_first.copy() # rating을 새로운 class로 수정 후 데이터

# 각 class의 비중을 다소 균등하게 조절하기 위해 신규 class로 묶어주기
dict_trans_class={1: ['AAA', 'AA+', 'AA', 'AA-', 'A+'],
                  2: ['A'],
                  3: ['A-'],
                  4: ['BBB+'],
                  5: ['BBB'],
                  6: ['BBB', 'BB+', 'BB'],
                  7: ['BB-', 'B+', 'B'],
                  8: ['B-', 'CCC', 'CCC-', 'C', 'D']
                  }

# 해당하는 기존 class를 신규 class로 변경
for class_main_idx in dict_trans_class:
  lst_class = dict_trans_class[class_main_idx]
  print(lst_class)
  for item in lst_class:
    lbl_final = np.where(lbl_first == item, class_main_idx, lbl_final)

['AAA', 'AA+', 'AA', 'AA-', 'A+']
['A']
['A-']
['BBB+']
['BBB']
['BBB', 'BB+', 'BB']
['BB-', 'B+', 'B']
['B-', 'CCC', 'CCC-', 'C', 'D']


In [28]:
lbl_final

array([1, 6, 6, 6, 1, 1, 1, 6, 1, 1, 1, 1, 2, 1, 6, 1, 1, 1, 6, 1, 6, 1,
       1, 1, 6, 6, 6, 4, 6, 2, 1, 1, 6, 4, 1, 1, 1, 1, 2, 6, 4, 1, 1, 2,
       1, 6, 6, 4, 6, 1, 1, 6, 6, 1, 1, 1, 2, 1, 6, 6, 6, 1, 1, 2, 6, 1,
       4, 2, 6, 6, 2, 6, 1, 2, 1, 2, 6, 6, 1, 1, 4, 1, 1, 1, 2, 4, 1, 2,
       4, 1, 1, 1, 1, 1, 4, 4, 1, 6, 1, 6], dtype=object)

In [29]:
# 신규 class 별 숫자를 확인
lbl_final_rest = lbl_final.copy()
uni_class = np.unique(lbl_final)
print(uni_class)
for idx,item in enumerate(uni_class):
    lbl_final_rest=np.where(lbl_final == item, idx, lbl_final_rest)
print(lbl_final_rest)
unique, counts = np.unique(lbl_final_rest, return_counts=True)
print(np.asarray((unique, counts)).T)

[1 2 4 6]
[[0 49]
 [1 12]
 [2 10]
 [3 29]]


In [32]:
# 더 진행하기 전에 기존 데이터 카피를 만들어주고, text를 토큰으로 받기
data_pre = data_main.copy()
data_token = data_pre["string_values"].values # type = np array

In [36]:
# text preprocessing

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [38]:
!pip install clean-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.2 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=d75ef3b77eca04388a3a3dd201b9b65dad70102f8421f778d104309778812c5a
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji


In [39]:


import numpy as np
import pandas as pd
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from cleantext import clean

In [40]:
# get lemmatized review
def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

# get stemmed review
def get_stemmed_text(corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

In [41]:
# remove all stopwords in english review
def remove_stop_words(corpus):
    english_stop_words = stopwords.words('english')
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split()
                      if word not in english_stop_words])
        )
    return removed_stop_words

In [43]:
# # preprocess review
def preprocess_txt(reviews):
    REPLACE_NO_SPACE = re.compile("(\.)|(!)\;)|(\:)|(\|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    NO_SPACE = ""
    SPACE = " "
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, reviews.lower())]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews

In [44]:
# clean review by cleantext libaray
def clean_text(sent):
  clean_sent=clean(sent,
      fix_unicode=True,               # fix various unicode errors
      to_ascii=True,                  # transliterate to closest ASCII representation
      lower=True,                     # lowercase text
      no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
      no_urls=True,                  # replace all URLs with a special token
      no_emails=True,                # replace all email addresses with a special token
      no_phone_numbers=True,         # replace all phone numbers with a special token
      no_numbers=False,               # replace all numbers with a special token
      no_digits=True,                # replace all digits with a special token
      no_currency_symbols=True,      # replace all currency symbols with a special token
      no_punct=True,                 # remove punctuations
      replace_with_punct="",          # instead of removing punctuations you may replace them
      replace_with_url="",
      replace_with_email="",
      replace_with_phone_number="",
      replace_with_number="",
      replace_with_digit="0",
      replace_with_currency_symbol="",
      lang="en" )
  return clean_sent

In [47]:
# 위에서 만든 전처리 함수들을 텍스트에 적용
data_pre["string_values"]=data_pre["string_values"].apply(lambda x:clean_text(x))
data_pre["string_values"]=data_pre["string_values"].apply(lambda x:preprocess_txt(x))
data_pre["string_values"]=data_pre["string_values"].apply(lambda x:remove_stop_words(x))
data_pre["string_values"]=data_pre["string_values"].apply(lambda x:get_stemmed_text(x))

In [49]:
#############################Tokenization

import matplotlib.pyplot as plt
text = data_token # data_token 은 string_values 열의 값을 np array로 받은 것
data_token_re=data_token.copy()
from keras.preprocessing.text import Tokenizer
MAX_NB_WORDS=500000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data_token_re)
data_seq = tokenizer.texts_to_sequences(data_token_re)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 5000
data_seq_pad = pad_sequences(data_seq, padding='pre', maxlen=maxlen)

In [ ]:
###############################Numeric part
from tensorflow.keras.utils import  to_categorical
lbl_binary = to_categorical(lbl_final_rest).astype(int)
# 4:101:mareket, 102:110:bond info, 111:157:finRatio
d1=dum
d2=data[list(data.iloc[:,4:101])]
d3=data[list(data.iloc[:,102:110])]
d4=data[list(data.iloc[:,111:157])]



from sklearn.preprocessing import MinMaxScaler
data_meta_normal1=d1.copy()
data_meta_normal2=d2.copy()
data_meta_normal3=d3.copy()
data_meta_normal4=d4.copy()


sce=MinMaxScaler()
for item_col in data_meta_normal2.columns:
    data_meta_normal2[item_col]=sce.fit_transform(d2[item_col].values.reshape(-1,1))

for item_col in data_meta_normal3.columns:
    data_meta_normal3[item_col]=sce.fit_transform(d3[item_col].values.reshape(-1,1))

for item_col in data_meta_normal4.columns:
    data_meta_normal4[item_col]=sce.fit_transform(d4[item_col].values.reshape(-1,1))




from tensorflow.keras.utils import to_categorical
lbl_binary=to_categorical(lbl_final_rest).astype(int)